# Aries Basic Controller Example
## Definitions Api

This can be used to write credential definitions to the ledger and search the definitions previously written by the agent.

A credential definition needs to be written to the ledger for every schema the agent wishes to issue. A credential definition acts as the public key in the [CL-signature scheme](https://wip-abramson.dev/cl-signatures) used in the sovrin ecosystem. 

For the interested the originally literature on this scheme can be found [here](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.186.5994&rep=rep1&type=pdf).

The key takeaway is that by using this type of signature scheme we are able to sign each attribute in the schema individual, making selective disclosure and efficient zero knowledge proof generation possible.

In [ ]:
%autoawait
import time
import asyncio

In [1]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    


## Write a Schema to the Ledger first

A credential definition is tied to a schema, each public DID may only write one credential definition per schema to the ledger. It is possible to write a credential definition for schemas that already exist on the ledger but because we are using a [fresh local network](http://localhost:9000) there aren't any so you must write one. See the schema_api notebook for more details.

In [2]:
# Define you schema name - must be unique on the ledger
schema_name = "my_new_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]

response = await agent_controller.schemas.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:my_new_schema:0.0.1


## Write a credential definition for this schema

#### Arguments
* schema_id - you need to pass in one of these and it must exist on the ledger
* tag - tag to identity the definition later, defaults to default
* support_revocation - Determines whether you will be able to later revoke the credential, defaults to False

**Note: You should be able to see the transactions on the local network [here](http://localhost:9000)**

In [3]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)


PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


## Get Created definition by id

As you can see it contains a bunch of BIG numbers all defined in the signature scheme. Notice that 'r' contains numbers for name, skill and master_secret, all attributes within the credential schema. (All credential schemas contain a master_secret to identify the holder)


In [4]:

definition = await agent_controller.definitions.get_by_id(cred_def_id)
print(definition)

{'credential_definition': {'ver': '1.0', 'id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schemaId': '10', 'type': 'CL', 'tag': 'default', 'value': {'primary': {'n': '87791508120866819316160844353443978035240850778035112494849784902439798913955266467744993072842182579264382299800668974775138311164882125825349580741986759653519796513997348025262575974206167697744477524764006366369599126064330403200391065019751283249613678884396874994544074971572171463369626807857195868232621533214699047844894289205249759435213716792863776836273521395448229862032859853816939156557824987008547094472535336669740044889888323166328984994745626570812073802496721296430132416789317801478861254962794307673674198246435880601861457823671401881736304700715182411704304392238258364863169590967182873698149', 's': '433626324637575427057286429083131185818295117726734853868748360245682078881862721416095706861199761251712295806735898985408403434384020648393630628970203036454988483158851183525050445908068810560120751452683

## Search all definitions the agent has created

#### Optional Arguments
* schema_id
* schema_issuer_did
* schema_version
* schema_name
* issuer_did
* cred_def_id

In [5]:
definitions = await agent_controller.definitions.search_created(cred_def_id=cred_def_id)
print(definitions)

{'credential_definition_ids': ['PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default']}
